In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from xgboost import XGBClassifier
import joblib
import matplotlib.pyplot as plt

In [2]:
file_path = 'GPS_Data_Simplified_2D_Feature_Map.xlsx'
df = pd.read_excel(file_path)

KeyboardInterrupt: 

In [ ]:
# Feature Engineering
def engineer_features(df):
    df['CP_diff'] = 0  # Placeholder for single prediction
    df['CP_consistency'] = 0  # Placeholder for single prediction
    df['signal_quality'] = df['CN0']
    df['PC_PIP_ratio'] = df['PC'] / df['PIP']
    df['EC_LC_ratio'] = df['EC'] / df['LC']
    df['CP_rate'] = 0  # Placeholder for single prediction
    df['PD_rate'] = 0  # Placeholder for single prediction
    df['quality_score'] = ((df['CN0'] >= 45.0) & (np.abs(df['PC'] - df['PIP']) < 1000.0)).astype(int)
    return df

In [ ]:
df = engineer_features(df)

In [ ]:
features = ['PRN', 'DO', 'PD', 'CP', 'EC', 'LC', 'PC', 'PIP', 'PQP', 'TCD', 'CN0',
            'CP_consistency', 'signal_quality', 'PC_PIP_ratio', 'EC_LC_ratio',
            'CP_rate', 'PD_rate', 'quality_score']
target = 'Output'

In [ ]:
X = df[features]
y = LabelEncoder().fit_transform(df[target])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
scaler = RobustScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
joblib.dump(scaler, 'scaler.pkl')

In [ ]:
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train_scaled, y_train)

In [ ]:
model.save_model('xgboost_gps_spoofing_model.json')


In [ ]:
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f'Testing Accuracy: {accuracy * 100:.2f}%')
print(classification_report(y_test, y_pred))


In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
display = ConfusionMatrixDisplay(conf_matrix)
display.plot()
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
plt.barh(features, model.feature_importances_)
plt.xlabel('Feature Importance')
plt.title('XGBoost Feature Importance')
plt.show()